In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

/Users/margheritarosnati/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/margheritarosnati/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# testing some code

In [2]:
name = '0a148697c.jpg'
labels_file =  "../data/labels/boxes_v2.csv"

In [4]:
y_raw = pd.read_csv(labels_file)
y_raw.head(2)

,ImageId,width,height,lt_x,lt_y,rb_x,rb_y
0,00003e153.jpg,0,0,0,0,0,0
1,0001124c7.jpg,0,0,0,0,0,0


In [ ]:
y_raw[y_raw.ImageId==name]

### first let's work out how to load the data into the model

In [6]:
ground_truths = y_raw[y_raw.ImageId == name]
array_of_coords = np.array(ground_truths[['lt_x', 'lt_y', 'rb_x', 'rb_y']])

In [15]:
n_boxes = array_of_coords.shape[0]
y_map = np.zeros((768, 768, n_boxes))
y_coord = np.zeros((768, 768, n_boxes*4))

for box_idx in range(n_boxes):
    # Loop over amount of boats per image ~ of order 10.
    box = array_of_coords[box_idx, :]
    # TODO: check that im using dimensions correctly here and agrees with static file
    y_map[box[0]:box[2], box[1]:box[3], box_idx] = 1
    y_coord[:,:,4*box_idx + 0] = round((box[2]+box[0])/2) # x-centre
    y_coord[:,:,4*box_idx + 1] = round((box[3]+box[1])/2) # y-centre
    y_coord[:,:,4*box_idx + 2] = box[2]-box[0] # width
    y_coord[:,:,4*box_idx + 3] = box[3]-box[1] # height
y_coord = list(map(int, y_coord))

In [26]:
def get_y_data(filename):
    '''
    Converts a filename into an array, with one channel for each boat in the image
    :param filename: filename string
    :return: np array of size [h, w, n_boats_in_this_image]
    '''

    ground_truths = y_raw[y_raw.ImageId == filename]
    array_of_coords = np.array(ground_truths[['lt_x', 'lt_y', 'rb_x', 'rb_y']])
    # array_of_coords is of shape [n_boxes, 4]

    n_boxes = array_of_coords.shape[0]
    y_map = np.zeros((768, 768, n_boxes))
    y_coord = np.zeros((768, 768, n_boxes*4))

    for box_idx in range(n_boxes):
        # Loop over amount of boats per image ~ of order 10.
        box = array_of_coords[box_idx, :]
        # TODO: check that im using dimensions correctly here and agrees with static file
        y_map[box[0]:box[2], box[1]:box[3], box_idx] = 1
        y_coord[:,:,4*box_idx + 0] = round((box[2]+box[0])/2) # x-centre
        y_coord[:,:,4*box_idx + 1] = round((box[3]+box[1])/2) # y-centre
        y_coord[:,:,4*box_idx + 2] = box[2]-box[0] # width
        y_coord[:,:,4*box_idx + 3] = box[3]-box[1] # height

    return y_map, y_coord

In [19]:
y_coord[0,0,:]

array([570., 492.,  26.,  38.,  76., 737., 150.,  44., 588., 463.,  26.,
        36., 726., 573.,  82., 132., 604., 433.,  23.,  30., 500., 620.,
        97., 140.])

In [28]:
def padder(list_of_arr):
    '''
    Pads each list of boat maps so all have the same depth (which is the max amount of
    boats across all images) and creates a numpy array of the result.
    :param list_of_arr: list of arrays, each shaped [768, 768, n_boats_in_this_image]
    :return: numpy array of shape [len(list_of_arr), h, w, maximum_n_boats]
    '''

    maximum_n_boats = max([x.shape[2] for x in list_of_arr])
    dim_arr = list_of_arr[0].shape

    n_box_max = maximum_n_boats

    b = np.zeros([len(list_of_arr), dim_arr[0], dim_arr[1], maximum_n_boats])
    for i, arr in enumerate(list_of_arr):
        b[i,:, :, :arr.shape[2]] = arr
    return b

In [21]:
files = [name]

In [30]:
y_data = padder([get_y_data(file)[0] for file in files])
y_corddata = padder([get_y_data(file)[1] for file in files])

In [31]:
y_corddata.shape

(1, 768, 768, 24)

### now let's get the anchors in the right shape

In [32]:
anchor_shapes = [(11, 11), (21, 21), (31, 31), (5, 11), (11, 21), (21, 31), (11, 5), (21, 11), (31, 21)]

In [2]:
anchor_shapes = [(11, 11), (21, 21)]

In [17]:
n_anchors = len(anchor_shapes)
y_anchors = np.zeros((768,768,n_anchors,4))
print(y_anchors.shape)
x = np.arange(768)
X, Y = np.meshgrid(x,x)
y_anchors[:,:,:,0] = np.reshape(X,(768,768,1))
y_anchors[:,:,:,1] = np.reshape(Y,(768,768,1))

print(y_anchors[:,:,1,2].shape)

(768, 768, 2, 4)
(768, 768)


In [28]:
n_anchors = len(anchor_shapes)
y_anchors = np.zeros((1,768,768,n_anchors,4))
print(y_anchors.shape)
x = np.arange(768)
X, Y = np.meshgrid(x,x)
y_anchors[:,:,:,:,0] = np.reshape(Y,(1,768,768,1))
y_anchors[:,:,:,:,1] = np.reshape(X,(1,768,768,1))
i=0
for anchor_shape in anchor_shapes:
    y_anchors[:,:,:,i,2] = np.ones((1,768,768))*anchor_shape[0] # width
    y_anchors[:,:,:,i,3] = np.ones((1,768,768))*anchor_shape[1] # length
    i +=1
print(y_anchors.shape)

(1, 768, 768, 2, 4)
(1, 768, 768, 2, 4)


In [26]:
# test
y_anchors[3,4,:,:]

array([[ 3.,  4., 11., 11.],
       [ 3.,  4., 21., 21.]])

In [69]:
tfanchors = tf.convert_to_tensor(y_anchors)

and voila! just a little tiling and we are done (see code)

### other random things

In [140]:
indx = np.random.randint(6,size=(2,2,4))
arr = np.random.randint(10,size=(2,2,6))
arr = np.arange(2*2*6).reshape((2,2,6))

In [141]:
print(indx[:,:,0], "\n\n")
for i in range(6):
    print(arr[:,:,i],"arr[:,:,i]",i)

[[1 2]
 [2 5]] 


[[ 0  6]
 [12 18]] arr[:,:,i] 0
[[ 1  7]
 [13 19]] arr[:,:,i] 1
[[ 2  8]
 [14 20]] arr[:,:,i] 2
[[ 3  9]
 [15 21]] arr[:,:,i] 3
[[ 4 10]
 [16 22]] arr[:,:,i] 4
[[ 5 11]
 [17 23]] arr[:,:,i] 5


In [146]:
ttry = indx[:,:,0]
print(ttry)
x = np.arange(2)
z = np.arange(6)
idx, idy = np.meshgrid(x,x)
np.array([[arr[0,0,ttry[0,0]],arr[0,1,ttry[0,1]]],[arr[1,0,ttry[1,0]], arr[1,1,ttry[1,1]]]])

[[1 2]
 [2 5]]


array([[ 1,  8],
       [14, 23]])

In [147]:
idxmr = [[0,0],[1,1]]
idymr = [[0,1],[0,1]]
idzmr = ttry
print(arr[idxmr,idymr,ttry], "\n\n", arr[idx,idy,ttry])


[[ 1  8]
 [14 23]] 

 [[ 1 14]
 [ 8 23]]


In [148]:
indx2 = np.random.randint(6,size=(2,2,4))

In [153]:
newidx = np.stack((idx,idx),axis=2)
newidy = np.stack((idy,idy),axis=2)


In [157]:
arr[newidx,newidy, indx[:,:,0:2]]

array([[[ 1,  0],
        [14, 12]],

       [[ 8,  8],
        [23, 23]]])

In [161]:
print(indx[:,:,0:2],"\n\n")
print(indx[:,:,0],"\n\n")
print(indx[:,:,1],"\n\n")
print(arr[idx,idy,indx[:,:,1]])

[[[1 0]
  [2 0]]

 [[2 2]
  [5 5]]] 


[[1 2]
 [2 5]] 


[[0 0]
 [2 5]] 


[[ 0 12]
 [ 8 23]]
